#### 9. mongodb에 크롤링 데이터 저장하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'

for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)
        
print (actors_info_list)

[{'배우이름': '하정우', '흥행지수': 22129, '출연영화': ['신과 함께-인과 연', '백두산', 'PMC: 더 벙커', '클로젯', '신과 함께-죄와 벌', '멋진 하루'], '랭킹': '1', '다른 이름': '김성훈; 河正宇', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영'}, {'배우이름': '마동석', '흥행지수': 19819, '출연영화': ['나쁜 녀석들: 더 무비', '신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물'], '랭킹': '2', '다른 이름': 'Ma Dongseok', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg'}, {'배우이름': '이병헌', '흥행지수': 16675, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '랭킹': '3', '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모자수집, 여행', '특기': '태권도, 스노우보드, 수영, 팔씨름'}, {'배우이름': '황정민', '흥행지수': 16119, '출연영화': ['공작', '인질', '다만 악

In [2]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.insert_many(actors_info_list)

In [3]:
actor_collection.delete_many({})